In [1]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

### Anticipatory Model  

In this file we develop the bidding model for an anticipatory agent. We mainly consider the following implications

* Adjusted flow
* Power Swap
* Overnomination

Our Nomination makes a difference on how much water can be discharged in the next step. On the other hand, we receive or have to deliver a power swap depending on the discrepancy between nomination and adjusted flow.  
The following formulas are directly included into the problem formulation:

$$
\begin{align*}
Q^\text{adj}_{r} &= \frac{Q^\text{nom}_{j,r} \cdot p_{j,r} + Q^\text{nom}_{O,r} \cdot p_{O,r}}{p_{j,r} + p_{O,r}} \\
P^\text{swap}_{j,r} &= p_{j,r} \cdot (Q^\text{nom}_r - Q^\text{adj}_r) - \sum\limits_{k \in \mathcal{K}_{O,r}} P^\text{Over}_k \\
P^\text{Over}_k  &= \max \{0, e_k \cdot (Q^\text{adj}_r - Q^\text{spill}_k)\} 
\end{align*}
$$

The Power Swap has to be delivered smoothly in the last 12 hours of the day. The Power Swaps in total add up and we have the following obligation:
$$
\begin{alignat*}{3}
y_{t} &= \sum\limits_{k \in \mathcal{K}} w_{k,t} + z^+_{t} - z^-_{t}  & & \text{ if } \; t = 1, \ldots, 12 \\ 
y_{t} &= \sum\limits_{k \in \mathcal{K}} w_{k,t} + 2 \cdot \sum\limits_{r \in \mathcal{R}} P^\text{swap}_r + z^+_{t} - z^-_{t} & & \text{ if } \; t = 13, \ldots, 24
\end{alignat*}
$$

Additionally, the adjusted flow poses the real average discharge constraint:
$$
T \cdot Q^\text{adj}_r = \sum\limits_{t = 1}^T Q^\text{real}_{r,t}
$$

### Define Parameters and Aggregated Other

We have to define an aggregated other user $O$ to reflect some constraints in our program.

In [31]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
R, plants, parts = read_data(filepath_Ljungan)
print() 

j = parts[1]
O = OtherParticipant(parts, j , R)[1]
K = [j.plants[1]]
K_O = [O.plants[1]]
pj = j.participationrate
pO = O.participationrate


println("Participation rate $(j.name): \n ", pj)
println("Participation rate $(O.name): \n ", pO)
println([k.spillreference for k in K_O])

Participation rate Sydkraft: 
 ________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Participation rate Other: 
 ________________________________
Flasjon  | 2.68    
Holsmjon | 2.68    

[1.4]


In [32]:
Stages = 2 # 2 lowest number as first stage is just to achieve nonanticipativity
T = 1
PPoints = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
I = length(PPoints)-1

println("Stages :" , Stages, "\n Hours: ", T, "\n I : ", I, "\n Price Points = ", [i for i in PPoints])

Qref = Dict{Reservoir, Float64}(r => 10.0 for r in R)
scenario_count = 1
Prices = [floor.(rand(T), sigdigits=3) for i in 1:scenario_count]
Inflows = [10.0]
Omega = [(price = p, inflow = v, nomination = 0.3) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]
# StartUp Costs
S = 0.1
# Cost for Up and Down Balancing
mu_up = 0.7
mu_down = 0.3

println("Uncertainty Set: ", Omega)
println(Qref)

Stages :2
 Hours: 1
 I : 5
 Price Points = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]


Uncertainty Set: NamedTuple{(:price, :inflow, :nomination), Tuple{Vector{Float64}, Float64, Float64}}[(price = [0.185]

, inflow = 10.0, nomination = 0.3)]
________________________________
Flasjon  | 10.0    
Holsmjon | 10.0    



In [33]:
function subproblem_builder_anticipatory(subproblem::Model, node::Int64)
    set_attribute(subproblem, "CPX_PARAM_EPINT", 1e-2)
    @variable(subproblem, 0 <= x[i = 1:I+1, t = 1:T] <= 50, SDDP.State, initial_value=0)
    @variable(subproblem, 0 <= l[r = R] <= r.maxvolume, SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, lind[r = R], SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, u_start[k = K], SDDP.State, initial_value = 0, Bin)
    @variable(subproblem, 0 <= Qnom[r = R] <= min([k.spillreference for k in filter(k -> k.reservoir == r, plants)]...), SDDP.State, initial_value = 0)
    @variable(subproblem, y[t=1:T] >= 0)
    @variable(subproblem, d[t=1:T, k = K], Bin)
    @variable(subproblem, u[t=1:T, k = K], Bin)
    @variable(subproblem, BALANCE_INDICATOR[r = R], Bin)
    @variable(subproblem, 0 <= w[t=1:T, k = K] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t=1:T] >= 0)
    @variable(subproblem, z_down[t=1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t=1:T, k = K] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t=1:T, r = R])
    @variable(subproblem, 0 <= Qadj[r = R])
    @variable(subproblem, Pswap[r = R])
    @variable(subproblem, Pover[k = K_O] >= 0)
    @variable(subproblem, f[r = R] >= 0)
    @variable(subproblem, s[r = R] >= 0)

    @constraint(subproblem, increasing[i = 1:I, t=1:T], x[i,t].out <= x[i+1,t].out)
    @constraint(subproblem, balance_ind[r = R], lind[r].out == lind[r].in - T * (Qnom[r].out - Qref[r])- s[r]) 
    @constraint(subproblem, nbal1[r = R], BALANCE_INDICATOR[r] => {Qnom[r].out <= Qref[r]})
    @constraint(subproblem, nbal2[r = R], !BALANCE_INDICATOR[r] => {0 <= lind[r].in})
    @constraint(subproblem, NoSpill[k = K], BALANCE_INDICATOR[k.reservoir] => {sum(Qnom[r_up].out for r_up in find_us_reservoir(k.reservoir)) <= k.spillreference})
    if node == 1
        @stageobjective(subproblem, 0)
        @constraint(subproblem, balance_transfer[r = R], l[r].out == l[r].in - T * Qnom[r].out - s[r]) 
    else
        @constraint(subproblem, adjustedflow[r = R], (pj[r] + pO[r]) * Qadj[r] - Qnom[r].in * pj[r] ==  pO[r])
        @constraint(subproblem, powerswap[r = R], Pswap[r] == pj[r] * (Qnom[r].in - Qadj[r]) - sum(Pover[k] for k in K_O))
        @constraint(subproblem, overnomination[k = K_O], Pover[k] >= k.equivalent * (Qadj[k.reservoir] - k.spillreference))
        @constraint(subproblem, endcond[k = K], u_start[k].out == u[T,k])
        @constraint(subproblem, startcond[k = K], u_start[k].in == u[1,k])
        @constraint(subproblem, clearing[t=1:T], y[t] == sum(1* x[i,t].in +  1* x[i+1,t].in for i in 1:I))
        @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
        @constraint(subproblem, obligation[t=1:T], y[t] - sum(Pswap[r] for r in R) == sum(w[t,k] for k in K) + z_up[t] - z_down[t])
        @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qadj[r] + f[r] * T - s[r])
        @constraint(subproblem, active[t=1:T, k=K], w[t,k] == u[t,k] * k.spillreference * k.equivalent)
        @constraint(subproblem, startup[t=1:T-1, k=K], d[t,k] >= u[t+1,k] - u[t,k])
        @constraint(subproblem, production[t=1:T, k=K], w[t,k] == Qeff[t,k] * k.equivalent)
        @constraint(subproblem, realwater[t=1:T, k=K], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
        @constraint(subproblem, spillwater[t=1:T, k=K], Qeff[t,k] <= k.spillreference)
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            for r in R
                JuMP.fix(f[r], om.inflow, force=true)
                JuMP.set_normalized_rhs(adjustedflow[r], pO[r] * om.nomination)
            end
            # Define Set of active variables for each hour
            I_t = Dict(t => 0 for t in 1:T)
            for t in 1:T
                for i in 1:I
                    if (om.price[t] >= PPoints[i]) && (om.price[t] <= PPoints[i+1])
                        I_t[t] = i
                    end
                end
            end
            # Include only active variables in stageobjective
            @stageobjective(subproblem ,sum(om.price[t] * y[t] -  mu_up * z_up[t] + mu_down * z_down[t]  - S * sum(d[t,k] for k in K) for t in 1:T))
            # Fix / Deactivate constraints by setting their coefficients to appropriate values or all zero.
            for t in 1:T
                for i in 1:I
                    if (i == I_t[t])
                        set_normalized_coefficient(clearing[t], x[i,t].in, -((om.price[t] - PPoints[i])/(PPoints[i+1] - PPoints[i])))
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, -((PPoints[i+1] - om.price[t])/(PPoints[i+1] - PPoints[i])))
                    else
                        set_normalized_coefficient(clearing[t], x[i,t].in, 0)
                        set_normalized_coefficient(clearing[t], x[i+1,t].in, 0)
                    end
                end
            end
        end
    end
    return
end

subproblem_builder_anticipatory (generic function with 1 method)

In [34]:
model_ant = SDDP.LinearPolicyGraph(
    subproblem_builder_anticipatory;
    stages = Stages,
    sense = :Max,
    upper_bound = 1e5,
    optimizer = CPLEX.Optimizer
)

SDDP.train(model_ant; iteration_limit = 10)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 2
  state variables : 13
  scenarios       : 1.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [47, 47]
  AffExpr in MOI.EqualTo{Float64}                                               : [4, 16]
  AffExpr in MOI.GreaterThan{Float64}                                           : [1, 1]
  AffExpr in MOI.LessThan{Float64}                                              : [5, 7]
  VariableRef in MOI.GreaterThan{Float64}                                       : [24, 25]
  VariableRef in MOI.LessThan{Float64}                                          : [13, 13]
  Variab

numerical stability report
  matrix range     [2e-01, 5e+00]
  objective range  [1e-01, 1e+00]
  bounds range     [2e-01, 1e+05]
  rhs range        [4e-01, 1e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-------------------------------------------------------------------


┌ Info: Writing cuts to the file `model.cuts.json`
└ @ SDDP C:\Users\Lenni\.julia\packages\SDDP\ppkik\src\algorithm.jl:287


ErrorException: Unable to retrieve solution from node 2.

  Termination status : INFEASIBLE
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION.

The current subproblem was written to `subproblem_2.mof.json`.

There are two common causes of this error:
  1) you have a mistake in your formulation, or you violated
     the assumption of relatively complete recourse
  2) the solver encountered numerical issues

See https://odow.github.io/SDDP.jl/stable/tutorial/warnings/ for more information.

In [35]:
sbproblem = JuMP.read_from_file("subproblem_2.mof.json")
print(sbproblem)

Max 0.185 y[1] - 0.7 z_up[1] + 0.3 z_down[1] - 0.1 d[1,Flasjo
] + x38
Subject to
 balance_ind[Flasjon
 ] : -lind[Flasjon
 ]_in + Qnom[Flasjon
 ]_out + s[Flasjon == 10
 ] + lind[Flasjon
 ]_out
 balance_ind[Holsmjon
 ] : -lind[Holsmjon
 ]_in + Qnom[Holsmjon
 ]_out + s[Holsmjon == 10
 ] + lind[Holsmjon
 ]_out
 adjustedflow[Flasjon
 ] : -1.84 Qnom[Flasjon
 ]_in + 4.5200000000000005 Qadj[Flasjon == 0.804
 ]
 adjustedflow[Holsmjon
 ] : 2.68 Qadj[Holsmjon
 ] == 0.804
 powerswap[Flasjon
 ] : -1.84 Qnom[Flasjon
 ]_in + 1.84 Qadj[Flasjon
 ] + Pover[Parteboda == 0
 ] + Pswap[Flasjon
 ]
 powerswap[Holsmjon
 ] : Pover[Parteboda
 ] + Pswap[Holsmjon == 0
 ]
 endcond[Flasjo
 ] : u_start[Flasjo
 ]_out - u[1,Flasjo == 0
 ]
 startcond[Flasjo
 ] : u_start[Flasjo
 ]_in - u[1,Flasjo == 0
 ]
 clearing[1] : -0.9249999999999999 x[1,1]_in + y[1] == 0
 nomination[Flasjon
 ] : Qreal[1,Flasjon
 ] - Qadj[Flasjon == 0
 ]
 nomination[Holsmjon
 ] : Qreal[1,Holsmjon
 ] - Qadj[Holsmjon == 0
 ]
 obligation[1] : y[1] - w[